### Взгляд на структуру json 

In [71]:
import json
import pandas as pd
import re

# бибдиотеки для работы с текстом
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain_core.documents import Document

# Укажите путь к вашему JSON-файлу
file_path = "../JSON/Печора/Исходные/Книга_4.json"

# Чтение JSON-файла
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

In [72]:
def extract_documents_recursive(sections, parent_title="", parent_level=0):
    """
    Рекурсивно извлекает документы из вложенных секций.
    :param sections: список секций.
    :param parent_title: заголовок родительской секции.
    :param parent_level: уровень родительской секции.
    :return: список объектов Document.
    """
    documents = []
    for section in sections:
        # Извлекаем текущую информацию
        title = section.get("title", "No Title")
        level = section.get("level", parent_level + 1)
        start_page = section.get("start_page", 0)
        end_page = section.get("end_page", 0)
        text = section.get("text", "")
        
        # Формируем метаданные
        metadata = {
            "title": title,
            "level": level,
            "parent_title": parent_title,
            "start_page": start_page,
            "end_page": end_page,
            'file': file_path.split('/')[-1]
        }
        
        # Создаём документ
        documents.append(Document(page_content=text, metadata=metadata))
        
        # Рекурсивно обрабатываем вложенные секции
        if "subsections" in section and isinstance(section["subsections"], list):
            documents.extend(
                extract_documents_recursive(
                    section["subsections"], parent_title=title, parent_level=level
                )
            )
    return documents

# Пример использования
documents = extract_documents_recursive(data)

# удаление только таблиц (на время)
def remove_table_and_save_titles(text):
    # Регулярное выражение для удаления содержимого таблиц, оставляя названия
    table_pattern = re.compile(
        r'((?:Таблица\s[\d.]+|Продолжение таблицы.*?)\s.*?)(?:\n.*?)+(?=(?:\n[А-Яа-я]|$))', 
        re.DOTALL
    )
    
    # Регулярное выражение для сохранения строк с рисунками (учитывает "рис.", "Рис.", "Рисунок")
    figure_pattern = re.compile(r'(?:рис\.|Рис\.|Рисунок)\s[\d.]+\s-.*?(?=\n|$)', re.IGNORECASE)
    
    # Удаляем содержимое таблиц, оставляя только их названия
    cleaned_text = table_pattern.sub(r'\1', text)
    
    # Убеждаемся, что строки с рисунками остаются неизменными
    figures = figure_pattern.findall(text)
    for figure in figures:
        if figure not in cleaned_text:
            cleaned_text += f"\n{figure}\n"

    return cleaned_text

for doc in documents:
    text = doc.page_content
    result = remove_table_and_save_titles(text)
    doc.page_content = result
    
# Формируем таблицу метаданных
metadata_table = pd.DataFrame([doc.metadata for doc in documents])
metadata_table.head(10)

,title,level,parent_title,start_page,end_page,file
0,ВВЕДЕНИЕ,1,,8,8,Книга_4.json
1,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,1,,8,16,Книга_4.json
2,1.1. Методика расчета водохозяйственных балансов,2,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,8,11,Книга_4.json
3,1.2. Водохозяйственное районирование бассейна ...,2,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,12,14,Книга_4.json
4,1.3. Водные ресурсы бассейна реки Печоры,2,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,15,15,Книга_4.json
5,1.4. Расчеты водохозяйственных балансов для ле...,2,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,16,16,Книга_4.json
6,1.5. Выводы,2,1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ,17,17,Книга_4.json
7,2. БАЛАНСЫ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ В ПОВЕРХНОСТНЫХ,1,,17,29,Книга_4.json
8,2.1 Методика расчета балансов ЗВ,2,2. БАЛАНСЫ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ В ПОВЕРХНОСТНЫХ,19,20,Книга_4.json
9,2.2 Расчет балансов ЗВ,2,2. БАЛАНСЫ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ В ПОВЕРХНОСТНЫХ,21,23,Книга_4.json


## Предобработка текста: стоит ли это делать?

### Самый главный вопрос: нужно ли это вообще?

Подход к предобработке текста зависит от используемого метода обработки данных:

1. **Методы типа TF-IDF или CountVectorizer:**
   - В таких случаях приведение текста к максимально простой форме (лемматизация, токенизация, удаление стоп-слов) действительно оправдано. Это улучшает качество признаков, сокращает размерность матрицы и помогает получить более точные результаты. Упрощенный текст позволяет фокусироваться на важной информации, исключая "шум".

2. **Модели типа BERT, FastText:**
   - Здесь ситуация иная. Эти модели обучены на текстах в их естественной форме и учитывают контекст, порядок слов и грамматические структуры. Приведение текста к упрощенной форме может ухудшить качество эмбеддингов, так как теряется информация, важная для понимания модели. Вместо этого стоит сосредоточиться на чистоте текста: 
     - Удалить лишние символы (`\n`, табуляции, спецсимволы).
     - Привести регистр текста к единообразию (например, в нижний регистр, если это нужно).
     - Сохранить стоп-слова и структуру текста.

Итак, выбор подхода зависит от используемого метода обработки текста и ваших целей.
Наша идея использовать модели типа `BERT`, поэтому обработка будет соответсвующей 


In [ ]:
# Функция для предобработки текста
def preprocess_page_content(text):
    # Удаляем лишние пробелы и символы табуляции
    text = re.sub(r"\s+", " ", text)
    # Приводим слова, написанные полностью в верхнем регистре, к заглавной первой букве
    text = re.sub(r"\b([А-ЯЁ]{2,})\b", lambda m: m.group(1).capitalize(), text)
    # Убираем спецсимволы, оставляя только буквы, цифры, пробелы и базовую пунктуацию
    text = re.sub(r"[^\w\s.,!?-–—()\"']", " ", text)
    # Очищаем лишние пробелы после всех операций
    return text.strip()

# Обрабатываем page_content для всех документов
for doc in documents:
    doc.page_content = preprocess_page_content(doc.page_content)

documents

[Document(metadata={'title': 'ВВЕДЕНИЕ', 'level': 1, 'parent_title': '', 'start_page': 8, 'end_page': 8, 'file': 'Книга_4.json'}, page_content='Введение В книге 4 «Схемы комплексного использования и охраны водных объектов, включая Ндв, бассейна реки Печоры» (далее Схема) представлены расчеты водохозяйственных балансов и балансов загрязняющих веществ, выполненные по расчетным и водохозяйственным участкам для лет различной водности (50 , 75  и 95  обеспеченности). Кроме того, в книге приведены расчеты величин пре  дельного изъятия воды из поверхностных водных объектов, которые были исполь  зованы в расчетах водохозяйственных балансов. Представленные материалы служат обоснованием необходимости выпол  нения водохозяйственных и водоохранных мероприятий для решения проблем водохозяйственного комплекса. 1. Водохозяйственные Балансы 1.1. Методика расчета водохозяйственных балансов В качестве методической основы расчета водохозяйственных балансов принята «Методика расчёта водохозяйственных бала

### Разбиение до фрагментов 1024 токена (под конкретную модель)

In [74]:
# часть кода для того чтобы понимать объем докуметов в токенах

# Загрузка токенизатора
model_name = "deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Функция для подсчета числа токенов
def get_max_token_document(documents):
    """
    Находит документ с наибольшим количеством токенов.
    :param documents: список объектов Document
    :return: документ с наибольшим числом токенов и длина этого текста в токенах
    """
    max_tokens = 0
    max_doc = None

    for doc in documents:
        # Токенизация текста документа
        tokens = tokenizer(doc.page_content, add_special_tokens=False)["input_ids"]
        num_tokens = len(tokens)
        print(f"Document: {doc.metadata.get('title', 'No Title')} has {num_tokens} tokens.")
        
        # Проверяем, больше ли текущий текст максимума
        if num_tokens > max_tokens:
            max_tokens = num_tokens
            max_doc = doc

    return max_doc, max_tokens

# Пример использования
max_doc, max_tokens = get_max_token_document(documents)
print(f"The document with the most tokens is: {max_doc.metadata.get('title', 'No Title')}, with {max_tokens} tokens.")
# максимальная длина токенов, обрабатываемых моделью
model = SentenceTransformer("deepvk/USER-bge-m3")
print("Max sequence length:", model.max_seq_length)

Document: ВВЕДЕНИЕ has 413 tokens.
Document: 1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ has 0 tokens.
Document: 1.1. Методика расчета водохозяйственных балансов has 435 tokens.
Document: 1.2. Водохозяйственное районирование бассейна реки Печоры has 99 tokens.
Document: 1.3. Водные ресурсы бассейна реки Печоры has 6 tokens.
Document: 1.4. Расчеты водохозяйственных балансов для лет различной водности has 614 tokens.
Document: 1.5. Выводы has 575 tokens.
Document: 2. БАЛАНСЫ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ В ПОВЕРХНОСТНЫХ has 0 tokens.
Document: 2.1 Методика расчета балансов ЗВ has 508 tokens.
Document: 2.2 Расчет балансов ЗВ has 1884 tokens.
Document: 2.3. Анализ результатов расчетов балансов ЗВ has 755 tokens.
Document: 2.4 Выводы has 538 tokens.
Document: Приложение 1 has 90 tokens.
The document with the most tokens is: 2.2 Расчет балансов ЗВ, with 1884 tokens.
Max sequence length: 8192


In [75]:
# Загрузка модели и токенизатора
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)
tokenizer = model.tokenizer

# Функция для разбивки текста на части с перекрытием
def split_text_with_overlap(text, max_tokens, overlap=100):
    """
    Разбивает текст на части, не превышающие max_tokens токенов, с перекрытием.
    Возвращает список частей.
    """
    tokens = tokenizer(text, add_special_tokens=False)["input_ids"]
    num_tokens = len(tokens)
    
    # Если текст уже меньше лимита, возвращаем как есть
    if num_tokens <= max_tokens:
        return [text]
    
    # Разбиваем текст с перекрытием
    chunks = []
    start = 0
    while start < num_tokens:
        end = min(start + max_tokens, num_tokens)
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        
        # Сдвигаем начало с учетом перекрытия
        start += max_tokens - overlap
    
    return chunks

# Функция для обработки списка документов
def process_documents_with_overlap(documents, max_tokens=1024, overlap=100):
    """
    Обрабатывает список документов. Если текст документа превышает max_tokens,
    разбивает его на части с перекрытием и создает новые документы с теми же метаданными.
    """
    processed_documents = []
    
    for doc in documents:
        # Получаем текст и токенизируем его
        text = doc.page_content
        metadata = doc.metadata
        
        # Разбиваем текст на части с перекрытием
        chunks = split_text_with_overlap(text, max_tokens, overlap)
        
        # Создаем новые документы для каждой части
        for idx, chunk in enumerate(chunks):
            new_metadata = metadata.copy()
            new_metadata["chunk_index"] = idx + 1  # Добавляем индекс части
            new_metadata["total_chunks"] = len(chunks)
            processed_documents.append(Document(metadata=new_metadata, page_content=chunk))
    
    return processed_documents

# Обработка документов с перекрытием
processed_documents = process_documents_with_overlap(documents)
processed_documents

[Document(metadata={'title': 'ВВЕДЕНИЕ', 'level': 1, 'parent_title': '', 'start_page': 8, 'end_page': 8, 'file': 'Книга_4.json', 'chunk_index': 1, 'total_chunks': 1}, page_content='Введение В книге 4 «Схемы комплексного использования и охраны водных объектов, включая Ндв, бассейна реки Печоры» (далее Схема) представлены расчеты водохозяйственных балансов и балансов загрязняющих веществ, выполненные по расчетным и водохозяйственным участкам для лет различной водности (50 , 75  и 95  обеспеченности). Кроме того, в книге приведены расчеты величин пре  дельного изъятия воды из поверхностных водных объектов, которые были исполь  зованы в расчетах водохозяйственных балансов. Представленные материалы служат обоснованием необходимости выпол  нения водохозяйственных и водоохранных мероприятий для решения проблем водохозяйственного комплекса. 1. Водохозяйственные Балансы 1.1. Методика расчета водохозяйственных балансов В качестве методической основы расчета водохозяйственных балансов принята «Ме

### Добавление метаданных 

In [76]:
# Добавляем порядковый номер в метаданные
for idx, doc in enumerate(processed_documents, start=1):
    doc.metadata['order'] = int(idx)
    doc.metadata['basin'] = 'Печора'

processed_documents

[Document(metadata={'title': 'ВВЕДЕНИЕ', 'level': 1, 'parent_title': '', 'start_page': 8, 'end_page': 8, 'file': 'Книга_4.json', 'chunk_index': 1, 'total_chunks': 1, 'order': 1, 'basin': 'Печора'}, page_content='Введение В книге 4 «Схемы комплексного использования и охраны водных объектов, включая Ндв, бассейна реки Печоры» (далее Схема) представлены расчеты водохозяйственных балансов и балансов загрязняющих веществ, выполненные по расчетным и водохозяйственным участкам для лет различной водности (50 , 75  и 95  обеспеченности). Кроме того, в книге приведены расчеты величин пре  дельного изъятия воды из поверхностных водных объектов, которые были исполь  зованы в расчетах водохозяйственных балансов. Представленные материалы служат обоснованием необходимости выпол  нения водохозяйственных и водоохранных мероприятий для решения проблем водохозяйственного комплекса. 1. Водохозяйственные Балансы 1.1. Методика расчета водохозяйственных балансов В качестве методической основы расчета водохоз

### Векторизация Qdrant

In [ ]:
# Сортировка документов перед загрузкой
processed_documents = sorted(processed_documents, key=lambda x: x.metadata['order'])

In [59]:
import uuid
import time
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

# 1. Подключение к Qdrant
client = QdrantClient(url="http://localhost:6333")

# 2. Загрузка модели SentenceTransformer
model_name = "deepvk/USER-bge-m3"  # Укажите модель, которая вам нужна
model = SentenceTransformer(model_name)

# 3. Проверяем, существует ли коллекция
collection_name = "testing_4_pechora"

try:
    # Проверяем существование коллекции
    collections = client.get_collections().collections  # Используем атрибут `.collections`
    if not any(c.name == collection_name for c in collections):
        # Если коллекция не существует, создаём новую
        vector_size = model.get_sentence_embedding_dimension()
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance="Cosine"),
        )
        print(f"Коллекция '{collection_name}' была создана.")
    else:
        print(f"Коллекция '{collection_name}' уже существует. Новые данные будут добавлены.")
except Exception as e:
    print(f"Ошибка при проверке коллекции: {e}")

# 4. Векторизация и загрузка новых данных в коллекцию
points = []
for doc in processed_documents:
    vector = model.encode(doc.page_content).tolist()
    point = PointStruct(
        id=str(uuid.uuid4()),  # Уникальный ID
        vector=vector,
        payload={"content": doc.page_content, **doc.metadata},  # Сохраняем текст и метаданные
    )
    points.append(point)

# Используем upsert для добавления данных в коллекцию
time.sleep(2)
try:
    client.upsert(collection_name=collection_name, points=points)
    print("Новые данные успешно добавлены в Qdrant!")
except Exception as e:
    print(f"Ошибка при добавлении данных: {e}")

Коллекция 'testing_4_pechora' была создана.
Новые данные успешно добавлены в Qdrant!


In [61]:
# Поиск документов
query = "В таблицах к рисунку приведены – перечень и основные характеристики расчетных водохозяй ственных участков, назначенных на реках бассейна (табл.1), характеристики ство ров, замыкающих водохозяйственные участки"
query_vector = model.encode(query).tolist()  # Векторизация запроса

# Выполняем поиск с использованием фильтрации
search_results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=2,  # Количество результатов
    query_filter={  # Используем правильное название параметра
        "must": [{"key": "basin", "match": {"value": "Печора"}}]
    },
)

print(f'Всего найдено: {len(search_results)}')
# Вывод результатов, включая контент
for result in search_results:
    print(f"ID: {result.id}")
    print(f"Score: {result.score}")
    print(f"Content: {result.payload['content']}")
    print(f"Metadata: {result.payload}")
    

Всего найдено: 2
ID: 4b125653-328f-4226-87cf-b58c9eba9eba
Score: 0.8601887
Content: 1.2. Водохозяйственное районирование бассейна реки Печоры Водохозяйственное районирование бассейна дано на рис.1. В таблицах к рисунку приведены – перечень и основные характеристики расчетных водохозяй  ственных участков, назначенных на реках бассейна (табл.1), характеристики ство  ров, замыкающих водохозяйственные участки (табл.2). Таблица 1 к рис.1
Metadata: {'content': '1.2. Водохозяйственное районирование бассейна реки Печоры Водохозяйственное районирование бассейна дано на рис.1. В таблицах к рисунку приведены – перечень и основные характеристики расчетных водохозяй  ственных участков, назначенных на реках бассейна (табл.1), характеристики ство  ров, замыкающих водохозяйственные участки (табл.2). Таблица 1 к рис.1', 'title': '1.2. Водохозяйственное районирование бассейна реки Печоры', 'level': 2, 'parent_title': '1. ВОДОХОЗЯЙСТВЕННЫЕ БАЛАНСЫ', 'start_page': 12, 'end_page': 14, 'file': 'Книга_4.json

### Создание коллекций с разными метриками поиска

In [32]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams
from sentence_transformers import SentenceTransformer

# 1. Подключение к Qdrant
client = QdrantClient(url="http://localhost:6333")

# 2. Загрузка модели
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)
vector_size = model.get_sentence_embedding_dimension()

# 3. Создание коллекций с разными метриками
metrics = ["Cosine", "Dot", "Euclid", "Manhattan"]
collection_names = {metric: f"documents_{metric.lower()}" for metric in metrics}

for metric, collection_name in collection_names.items():
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance=metric),
    )

# 4. Загрузка данных в каждую коллекцию
points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=model.encode(doc.page_content).tolist(),
        payload={"content": doc.page_content, **doc.metadata},
    )
    for doc in processed_documents
]

for metric, collection_name in collection_names.items():
    client.upsert(collection_name=collection_name, points=points)

print("Documents successfully uploaded to all collections!")

/var/folders/rf/r9_1j8lj3m72m2lv3v1tp3rh0000gn/T/ipykernel_1389/2795548259.py:18: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Documents successfully uploaded to all collections!


In [35]:
query = "О чем говорится в Таблица 5.63? "
query_vector = model.encode(query).tolist()

# Поиск в каждой коллекции
for metric, collection_name in collection_names.items():
    print(f"Searching in {metric} collection...")
    search_results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=5,
    )
    for result in search_results:
        print(f"ID: {result.id}")
        print(f"Score: {result.score}")
        print(f"Content: {result.payload['content']}")
    print("-" * 40)

Searching in Cosine collection...
ID: e351524f-00bd-4edf-a8f1-1f618ce4989e
Score: 0.58543724
Content: Продолжение таблицы 5.25
ID: 7fc3ec77-baae-4600-8f6c-78ad82ba4dfc
Score: 0.58543724
Content: Продолжение таблицы 5.25
ID: b622549f-1458-4515-b130-23a517d24459
Score: 0.49149868
Content: Таблица 5.63   Основные характеристики гидроузлов Рисунок 5.50   Площадь акватории Цимлянского водохранилища при различных отметках у Рисунок 5.51   Забор воды на цели прудового рыбоводства в бассейне р.Дон
ID: e515229e-c3a2-4c27-8363-b1f46a4d7699
Score: 0.47579733
Content: Продолжение таблицы 3.4
ID: cade3acf-c8ef-4d05-b5f4-2c4c06d0f155
Score: 0.44613242
Content: 5.5.2 Водоснабжение Использование водных ресурсов для водоснабжения городов и промышленных предприятий, включая Тэс и Аэс, а также сельских населенных пунктов и обводнения пастбищ на всех расчетных уровнях, имея высокую обеспеченность (Р   95 ), отличается относительно других участников водохозяйственного комплекса, незначительной величиной бе

### Рабочий хост

In [78]:
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer

# Подключение к удалённому серверу Qdrant
client = QdrantClient(url="http://192.168.137.253:6333")

# Модель для векторизации
model_name = "deepvk/USER-bge-m3"
model = SentenceTransformer(model_name)

# Имя коллекции
collection_name = "testing_4_pechora"

# Проверка или создание коллекции
collections = client.get_collections().collections
if not any(c.name == collection_name for c in collections):
    vector_size = model.get_sentence_embedding_dimension()
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance="Cosine"),
    )
    print(f"Коллекция '{collection_name}' создана.")
else:
    print(f"Коллекция '{collection_name}' уже существует.")

points = []
for doc in processed_documents:
    vector = model.encode(doc.page_content).tolist()
    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload={"content": doc.page_content, **doc.metadata},
    )
    points.append(point)


# Загрузка данных
client.upsert(collection_name=collection_name, points=points)
print("Данные успешно добавлены.")

Коллекция 'testing_4_pechora' уже существует.
Данные успешно добавлены.
